In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
import csv
import time
import re
import urllib
import json
import numpy as np
import threading
from tqdm import tqdm
import pickle
import os

In [2]:
file_list = os.listdir(path='data_link')

In [3]:
file_list[0].split('_')

['link', 'data', 'stack', 'overflow', 'python', '24', '32.pickle']

In [4]:
file_list_sort = []
for i_i in range(int(len(file_list)/2)):
    for i_j in range(len(file_list)):
        if (file_list[i_j].split('_')[0] == 'link') and (file_list[i_j]).split('_')[5] == str(8*i_i):
            file_list_sort.append(file_list[i_j])
for i_i in range(int(len(file_list)/2)):
    for i_j in range(len(file_list)):
        if (file_list[i_j].split('_')[0] == 'status') and (file_list[i_j]).split('_')[6] == str(8*i_i):
            file_list_sort.append(file_list[i_j])

In [5]:
file_list_sort

['link_data_stack_overflow_python_0_8.pickle',
 'link_data_stack_overflow_python_8_16.pickle',
 'link_data_stack_overflow_python_16_24.pickle',
 'link_data_stack_overflow_python_24_32.pickle',
 'link_data_stack_overflow_python_32_40.pickle',
 'link_data_stack_overflow_python_40_48.pickle',
 'link_data_stack_overflow_python_48_56.pickle',
 'link_data_stack_overflow_python_56_64.pickle',
 'link_data_stack_overflow_python_64_72.pickle',
 'link_data_stack_overflow_python_72_80.pickle',
 'status_link_data_stack_overflow_python_0_8.pickle',
 'status_link_data_stack_overflow_python_8_16.pickle',
 'status_link_data_stack_overflow_python_16_24.pickle',
 'status_link_data_stack_overflow_python_24_32.pickle',
 'status_link_data_stack_overflow_python_32_40.pickle',
 'status_link_data_stack_overflow_python_40_48.pickle',
 'status_link_data_stack_overflow_python_48_56.pickle',
 'status_link_data_stack_overflow_python_56_64.pickle',
 'status_link_data_stack_overflow_python_64_72.pickle',
 'status_lin

In [6]:
# all_link_form_file = np.array([])
# all_status_form_file = np.array([])

In [7]:
# file_all = file_list_sort
# link_colab = 'data_link/'
# count = 0
# for num_file in range(int(len(file_all)/2)):
#     with open(link_colab+file_all[num_file], "rb") as f:
#         load_file_link = pickle.load(f)#
#         f_link = open(link_colab+file_all[num_file+10],'rb')
#         load_file_status = pickle.load(f_link)#
#         f_link.close()
#         print(load_file_link.shape[0])
#         print(load_file_status.shape[0])
#         print("--------------")
#         for num_link in range(load_file_link.shape[0]):
#             if load_file_link[num_link] not in all_link_form_file:
#                 # all_link_form_file[count] = load_file_link[num_link]
#                 # all_status_form_file[count] = load_file_status[num_link]
#                 # all_link_form_file = np.append(all_link_form_file,load_file_link[num_link])
#                 # all_status_form_file = np.append(all_status_form_file,load_file_status[num_link])
#                 all_link_form_file.append(load_file_link[num_link])
#                 all_status_form_file.append(load_file_status[num_link])
#                 count += 1
#             if count % 2500 == 0:
#                 print(count)
# print(count)


In [8]:
all_link_status_form_file = dict()

In [9]:
file_all = file_list_sort
link_colab = 'data_link/'
count = 0
for num_file in range(int(len(file_all)/2)):
    with open(link_colab+file_all[num_file], "rb") as f:
        load_file_link = pickle.load(f)#
        f_link = open(link_colab+file_all[num_file+10],'rb')
        load_file_status = pickle.load(f_link)#
        f_link.close()
        print(load_file_link.shape[0])
        print(load_file_status.shape[0])
        print("--------------")
        for num_link in range(load_file_link.shape[0]):
            all_link_status_form_file.update({str(load_file_link[num_link]):list(load_file_status[num_link])})
            count += 1
        if count % 2500 == 0:
            print(count)
print(count)


218400
218400
--------------
218400
218400
--------------
218400
218400
--------------
218400
218400
--------------
218400
218400
--------------
218250
218250
--------------
218000
218000
--------------
217429
217429
--------------
218000
218000
--------------
217339
217339
--------------
2181018


In [10]:
load_file_link = None
load_file_status = None

In [11]:
len(all_link_status_form_file)

1981285

In [12]:
# all_link_form_file_array = np.array(all_link_status_form_file.keys())
# all_status_form_file_array = np.array(all_link_status_form_file.values())

In [13]:
max_pages = len(all_link_status_form_file)
count_thread = 10
num_per_thread = max_pages//count_thread
page_last = max_pages%count_thread
all_thread = [None]*count_thread
all_link_form_file = [None]*count_thread
all_status_form_file = [None]*count_thread

In [14]:
# all_link_form_file_array = np.array([])
# all_status_form_file_array = np.array([])

In [15]:
def add_data(all_link_form_file,all_status_form_file,data_link,start,stop,num_thread):
    count = 0
    all_link = []
    all_status = []
    for k,v in data_link.items():
        if count < start:
            count += 1
            continue
        elif count >= stop:
            break
        all_link.append(k)
        all_status.append(v)
        count += 1
        if count % 2500 == 0:
            print(count)
    all_status_form_file[num_thread] = all_status
    all_link_form_file[num_thread] = all_link

In [16]:
li = (page_last - 1)
for i in range(count_thread):
    if i < page_last:
        all_thread[i] = threading.Thread(target=add_data,args=(all_link_form_file,all_status_form_file,all_link_status_form_file,i*num_per_thread + i*(1),(i+1)*num_per_thread + (i+1)*1,i))
        print(i*num_per_thread + i*(1),(i+1)*num_per_thread + (i+1)*1,i)
    elif page_last == 0:
        all_thread[i] = threading.Thread(target=add_data,args=(all_link_form_file,all_status_form_file,all_link_status_form_file,i*num_per_thread,(i+1)*num_per_thread,i))
        print(i*num_per_thread,(i+1)*num_per_thread,i)
    else:
        all_thread[i] = threading.Thread(target=add_data,args=(all_link_form_file,all_status_form_file,all_link_status_form_file,i*num_per_thread + (li+1)*(1),(i+1)*num_per_thread + (li+1)*1,i))
        print(i*num_per_thread + (li+1)*(1),(i+1)*num_per_thread + (li+1)*1,i)

0 198129 0
198129 396258 1
396258 594387 2
594387 792516 3
792516 990645 4
990645 1188773 5
1188773 1386901 6
1386901 1585029 7
1585029 1783157 8
1783157 1981285 9


In [17]:
for i in range(count_thread):
    all_thread[i].start()
for i in range(count_thread):
    all_thread[i].join()

2500
5000
7500
10000
12500
15000
17500
20000
22500
25000
27500
30000
32500
35000
37500
40000
42500
45000
47500
50000
52500
55000
57500
60000
62500
65000
67500
70000
72500
75000
77500
80000
82500
85000
87500
90000
92500
95000
97500
100000
102500
105000
107500
110000
112500
115000
117500
120000
122500
125000
127500
130000
132500
135000
137500
140000
142500
145000
147500
150000
152500
155000
157500
200000
202500
205000
207500
210000
212500
215000
217500
220000
222500
225000
227500
230000
232500
235000
237500
240000
242500
245000
247500
250000
252500
255000
257500
260000
262500
265000
267500
270000
272500
275000
277500
280000
282500
285000
287500
290000
292500
295000
297500
300000
302500
305000
307500
160000
162500
165000
167500
170000
172500
175000
177500
180000
182500
185000
187500
190000
192500
195000
197500
397500
400000
402500
405000
407500
410000
412500
415000
417500
310000
312500
315000
317500
320000
322500
325000
327500
330000
332500
420000
422500
425000
427500
430000
432500
435000

In [18]:
all_link_status_form_file = None

In [26]:
all_link_form_file_array = np.array([])
all_status_form_file_array = np.array([])

In [27]:
for i in all_link_form_file:
    all_link_form_file_array = np.append(all_link_form_file_array,np.array(i))
# for i in all_status_form_file:
#     array_i = np.array(i)
#     l = array_i.shape[0]
#     all_status_form_file_array = np.append(all_status_form_file_array,array_i.reshape((l,3)))
    

In [28]:
all_link_form_file_array.shape

(1981285,)

In [35]:
len(set(all_link_form_file_array))

1981285

In [22]:
all_status_form_file_array = all_status_form_file_array.reshape((all_link_form_file_array.shape[0],3))

In [23]:
all_status_form_file_array.shape

(1981285, 3)

In [24]:
all_status_form_file_array[0]

array(['12838', '51', '3.3m'], dtype='<U32')

In [30]:
with open(f"link_data_stack_overflow_python_{all_link_form_file_array.shape[0]}.pickle", "wb") as f:
    pickle.dump(all_link_form_file_array, f, protocol=pickle.HIGHEST_PROTOCOL)

In [25]:
with open(f"status_data_stack_overflow_python_{all_status_form_file_array.shape[0]}.pickle", "wb") as f:
    pickle.dump(all_status_form_file_array, f, protocol=pickle.HIGHEST_PROTOCOL)

In [13]:
with open(f"status_link_data_stack_overflow_python_{all_link_status_form_file}.pickle", "wb") as f:
    json.dump(all_link_status_form_file,f)
    #pickle.dump(all_link_status_form_file, f, protocol=pickle.HIGHEST_PROTOCOL)